# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [35]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pandas as pd

browser = webdriver.Chrome(executable_path='chromedriver\chromedriver.exe')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2019')
browser.find_element_by_id('ctl05_btnQuery').click()

time.sleep(3)

html_source = browser.page_source
html_source

soup = BeautifulSoup(html_source, 'html.parser')
trs = soup.find('table', class_='TABLE_G').find_all('tr')

column = trs[0]
allRows = trs[1:-3]
rowspan = []

headers = [header.get_text() for header in column.find_all('th')]
results = [[data.get_text() for data in row.find_all('td')] for row in allRows]

for no, tr in enumerate(allRows):
    # print(no, tr)
    for td_no, td_data in enumerate(tr.find_all('td')):
        # print(td_no, td_data.has_attr("rowspan"), td_data)
        if td_data.has_attr("rowspan"):
            rowspan.append((no, td_no, int(td_data["rowspan"]), td_data.get_text()))
# print(rowspan)    
if rowspan:
    for i in rowspan:
        # print(i)
        for j in range(1, i[2]):
            # - Add value in next tr.
            results[i[0]+j].insert(i[1], i[3])
# print(results)

df = pd.DataFrame(data=results, columns=headers)
df


,監測項目,單位,監測日期,監測值,標註
0,SO2,ppb,2019/01,1.80,
1,SO2,ppb,2019/02,1.60,
2,SO2,ppb,2019/03,1.90,
3,SO2,ppb,2019/04,2.20,
4,SO2,ppb,2019/05,1.70,
...,...,...,...,...,...
72,NO2,ppb,2019/07,11.06,
73,NO2,ppb,2019/08,7.73,
74,NO2,ppb,2019/09,8.46,
75,NO2,ppb,2019/10,7.88,


In [36]:
df[:8]

,監測項目,單位,監測日期,監測值,標註
0,SO2,ppb,2019/01,1.80,
1,SO2,ppb,2019/02,1.60,
2,SO2,ppb,2019/03,1.90,
3,SO2,ppb,2019/04,2.20,
4,SO2,ppb,2019/05,1.70,
5,SO2,ppb,2019/06,1.90,
6,SO2,ppb,2019/07,2.10,
7,SO2,ppb,2019/08,2,


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [34]:
df.loc[(df['監測項目'].isin(['SO2', 'CO'])) & (df['監測日期'].isin(['2019/01', '2019/02','2019/03','2019/04','2019/05','2019/06','2019/07','2019/08']))]
browser.quit()

,監測項目,單位,監測日期,監測值,標註
0,SO2,ppb,2019/01,1.80,
1,SO2,ppb,2019/02,1.60,
2,SO2,ppb,2019/03,1.90,
3,SO2,ppb,2019/04,2.20,
4,SO2,ppb,2019/05,1.70,
5,SO2,ppb,2019/06,1.90,
6,SO2,ppb,2019/07,2.10,
7,SO2,ppb,2019/08,2,
11,CO,ppm,2019/01,0.40,
12,CO,ppm,2019/02,0.43,
